# Week 9 Coding Quiz 

In [4]:
!pip install statsmodels 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 41.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import skew

# Reproducibility
np.random.seed(0)

print('Libraries imported. Seed set to 0.')

Libraries imported. Seed set to 0.


## Simulation function

In [11]:
def simulate(A=1, B=1, C=10, D=1000):
    """
    Simulate data for Y, X, W.
    Parameters:
        A: True coefficient on X
        B: Noise level in X (variance of U in X=W+U where U~N(0,B))
        C: Noise level in Y (std dev of the noise term)
        D: Sample size
    Returns:
        Y, X, W arrays
    """
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

Y, X, W = simulate()
len(Y), len(X), len(W)

(1000, 1000, 1000)

## Question 1
Which of the following is closest to the probability of detecting a nonzero effect of ﻿X﻿ on ﻿Y﻿ (the t-value of ﻿X﻿ is greater in absolute value than about 1.96) given A = 1, B = 1, C = 10, D = 1000? Include W in the regression.

In [7]:
def detection_probability(A=1, B=1, C=10, D=1000, reps=1000, alpha=0.05):
    crit = 1.96  # approx two-sided 5% threshold
    t_vals = []
    for _ in range(reps):
        Y, X, W = simulate(A=A, B=B, C=C, D=D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_vals.append(model.tvalues[1])  # t for X
    return np.mean(np.abs(t_vals) > crit)

prob_detect_q1 = detection_probability(A=1, B=1, C=10, D=1000, reps=1000)
print(f"Q1 — Probability of detection: {prob_detect_q1:.3f}")

Q1 — Probability of detection: 0.878


## Question 2
Which of the following is closest to the skew of the estimate in that case? (You can compute this using scipy.)


In [12]:
from scipy.stats import skew
import numpy as np
import statsmodels.api as sm

def simulate(A=1, B=1, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

coefs = []
for _ in range(1000):
    Y, X, W = simulate(A=1, B=1, C=10, D=1000)
    XW = sm.add_constant(np.column_stack([X, W]))
    model = sm.OLS(Y, XW).fit()
    coefs.append(model.params[1])  # coefficient on X

coef_skew = skew(coefs)
print("Skew of the estimate:", coef_skew)


Skew of the estimate: 0.014365290670855346


## Question 3
With A = 1, C = 10, D = 1,000, what value of B is needed to detect that the Data Generating Process (DGP) has a nonzero coefficient for X about 50% of the time? (Choose the closest value.)

In [13]:
def power_for_B(B, reps=500):
    return detection_probability(A=1, B=B, C=10, D=1000, reps=reps)

Bs = [0.2, 0.6, 1.8, 5.4]
powers_B = {B: power_for_B(B) for B in Bs}
powers_B

{0.2: np.float64(0.102),
 0.6: np.float64(0.464),
 1.8: np.float64(1.0),
 5.4: np.float64(1.0)}

## Question 4
With B = 1, C = 10, D = 100 (note the different value of D), what value of A is needed to detect that the DGP has a nonzero coefficient for X about 50% of the time? (Choose the closest value.) 

In [14]:
def power_for_A(A, reps=500):
    return detection_probability(A=A, B=1, C=10, D=100, reps=reps)

As = [0.5, 1.0, 2.0, 4.0]
powers_A = {A: power_for_A(A) for A in As}
powers_A

{0.5: np.float64(0.084),
 1.0: np.float64(0.156),
 2.0: np.float64(0.518),
 4.0: np.float64(0.986)}

## Week 9 Homework Reflection

1. Write some code that will use a simulation to estimate the standard deviation of the coefficient when there is heteroskedasticity.  
Compare these standard errors to those found via statsmodels OLS or a similar linear regression model.

In [15]:
import numpy as np
import statsmodels.api as sm

rng = np.random.default_rng(0)

def sim_hetero_once(n=1000, beta=1.0, sigma=1.0, h=1.5):
    """
    DGP: Y = beta*X + e, with heteroskedastic errors: Var(e|X) = sigma^2*(1 + h*|X|)^2
    """
    X = rng.normal(0, 1, n)
    e = rng.normal(0, sigma * (1 + h * np.abs(X)))
    Y = beta * X + e
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1).fit()
    # HC1 robust
    model_hc1 = model.get_robustcov_results(cov_type="HC1")
    return {
        "beta_hat": model.params[1],
        "se_conventional": model.bse[1],
        "se_hc1": model_hc1.bse[1]
    }

def sim_hetero_many(R=1000, n=1000, beta=1.0, sigma=1.0, h=1.5):
    out = [sim_hetero_once(n=n, beta=beta, sigma=sigma, h=h) for _ in range(R)]
    bhats = np.array([d["beta_hat"] for d in out])
    se_conv = np.array([d["se_conventional"] for d in out])
    se_hc1  = np.array([d["se_hc1"] for d in out])
    return {
        "emp_sd_beta_hat": bhats.std(ddof=1),
        "mean_se_conventional": se_conv.mean(),
        "mean_se_hc1": se_hc1.mean()
    }

res_het = sim_hetero_many(R=1000, n=1000, beta=1.0, sigma=1.0, h=1.5)
res_het


{'emp_sd_beta_hat': np.float64(0.10673306927024596),
 'mean_se_conventional': np.float64(0.07510474786708725),
 'mean_se_hc1': np.float64(0.111413843051685)}

## Heteroskedasticity
I ran a simulation where the error variance got bigger as 
𝑋
X increased. When I compared the regular OLS standard errors to the robust ones (HC1), the regular ones were too small. The HC1 robust errors matched the actual spread of the simulated coefficients much better. Basically, when the noise changes with 
𝑋
X, you need robust SEs or you’ll underestimate uncertainty.

2. Write some code that will use a simulation to estimate the standard deviation of the coefficient when errors are highly correlated / non-independent.
Compare these standard errors to those found via statsmodels OlS or a similar linear regression model.

In [16]:
import numpy as np
import statsmodels.api as sm

rng = np.random.default_rng(1)

def sim_cluster_once(G=100, m=10, beta=1.0, tau=1.2, sigma=1.0):
    """
    DGP: G clusters, m obs per cluster (n = G*m).
    X ~ N(0,1); errors have cluster shock u_g and idiosyncratic v.
    Y = beta*X + e,  e = u_g + v
    """
    n = G * m
    g_id = np.repeat(np.arange(G), m)
    X = rng.normal(0, 1, n)
    u = rng.normal(0, tau, G)  # cluster shocks
    v = rng.normal(0, sigma, n)
    e = u[g_id] + v
    Y = beta * X + e
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1).fit()
    # Cluster-robust by group id
    model_clu = model.get_robustcov_results(cov_type="cluster", groups=g_id)
    return {
        "beta_hat": model.params[1],
        "se_conventional": model.bse[1],
        "se_cluster": model_clu.bse[1]
    }

def sim_cluster_many(R=1000, G=100, m=10, beta=1.0, tau=1.2, sigma=1.0):
    out = [sim_cluster_once(G=G, m=m, beta=beta, tau=tau, sigma=sigma) for _ in range(R)]
    bhats = np.array([d["beta_hat"] for d in out])
    se_conv = np.array([d["se_conventional"] for d in out])
    se_clu  = np.array([d["se_cluster"] for d in out])
    return {
        "emp_sd_beta_hat": bhats.std(ddof=1),
        "mean_se_conventional": se_conv.mean(),
        "mean_se_cluster": se_clu.mean()
    }

res_cluster = sim_cluster_many(R=1000, G=100, m=10, beta=1.0, tau=1.2, sigma=1.0)
res_cluster


{'emp_sd_beta_hat': np.float64(0.047369696301241354),
 'mean_se_conventional': np.float64(0.04925726002263184),
 'mean_se_cluster': np.float64(0.04872828437530312)}

## Correlated errors
I simulated data where observations were grouped and shared the same random shock (so their errors were correlated). The regular OLS SEs again came out too low, while the cluster-robust SEs were much closer to the true variation. When I tried a simple bootstrap that ignored the group structure, it also underestimated the SE. So if errors are related, you have to use cluster-robust or block bootstrap methods to get it right.

Show that if the correlation between coefficients is high enough, then the estimated standard deviation of the coefficient, using bootstrap errors, 
might not match that found by a full simulation of the Data Generating Process.  (This can be fixed if you have a huge amount of data for the bootstrap simulation.)

In [19]:
import numpy as np, statsmodels.api as sm
rng = np.random.default_rng(0)

def dgp_cluster(G=40, m=8, beta=1.0, tau=3.0, sigma=1.0):
    """Strong intra-cluster correlation: rho = tau^2 / (tau^2 + sigma^2) ≈ 0.9."""
    n = G*m
    g = np.repeat(np.arange(G), m)
    X = rng.normal(0, 1, n)
    u = rng.normal(0, tau, G)         # cluster shock
    v = rng.normal(0, sigma, n)       # idiosyncratic
    e = u[g] + v
    Y = beta*X + e
    return Y, X, g

def iid_resid_boot_se(Y, X, B=800):
    X1 = sm.add_constant(X)
    base = sm.OLS(Y, X1).fit()
    resid = base.resid
    betas = []
    for _ in range(B):
        e_star = rng.choice(resid, size=len(resid), replace=True)  # WRONG under dependence
        Y_star = base.fittedvalues + e_star
        betas.append(sm.OLS(Y_star, X1).fit().params[1])
    return np.std(betas, ddof=1)

def cluster_boot_se(Y, X, groups, B=800):
    # Resample entire clusters (block bootstrap)
    X1 = sm.add_constant(X)
    base = sm.OLS(Y, X1).fit()
    betas = []
    gids = np.unique(groups)
    idx_by_g = {g: np.where(groups==g)[0] for g in gids}
    for _ in range(B):
        sample_g = rng.choice(gids, size=len(gids), replace=True)
        idx = np.concatenate([idx_by_g[g] for g in sample_g])
        betas.append(sm.OLS(Y[idx], X1[idx]).fit().params[1])
    return np.std(betas, ddof=1)

def empirical_sd_over_dgp(R=800, **dgp_kwargs):
    bhats = []
    for _ in range(R):
        Y, X, g = dgp_cluster(**dgp_kwargs)
        bhats.append(sm.OLS(Y, sm.add_constant(X)).fit().params[1])
    return np.std(bhats, ddof=1)

G, m = 40, 8
Y, X, g = dgp_cluster(G=G, m=m)
emp_sd = empirical_sd_over_dgp(R=800, G=G, m=m)
se_iid  = iid_resid_boot_se(Y, X, B=800)
se_clust= cluster_boot_se(Y, X, g, B=800)

{"empirical_SD_DGP": emp_sd, "SE_naive_IID_boot": se_iid, "SE_cluster_boot": se_clust}

{'empirical_SD_DGP': np.float64(0.16722904846233982),
 'SE_naive_IID_boot': np.float64(0.15280290929995982),
 'SE_cluster_boot': np.float64(0.18480915168299267)}

In [20]:
# Larger sample: more clusters and/or size per cluster
G, m = 400, 20
Y_big, X_big, g_big = dgp_cluster(G=G, m=m)
emp_sd_big = empirical_sd_over_dgp(R=400, G=G, m=m)
se_iid_big  = iid_resid_boot_se(Y_big, X_big, B=400)
se_clust_big= cluster_boot_se(Y_big, X_big, g_big, B=400)

{"empirical_SD_DGP_big": emp_sd_big, "SE_naive_IID_boot_big": se_iid_big, "SE_cluster_boot_big": se_clust_big}


{'empirical_SD_DGP_big': np.float64(0.03560541140860346),
 'SE_naive_IID_boot_big': np.float64(0.039213296878995545),
 'SE_cluster_boot_big': np.float64(0.03751764290581541)}

When errors are highly correlated, a naïve IID residual bootstrap underestimates the standard error compared to a full DGP simulation. A cluster/block bootstrap (or cluster-robust SEs) matches the empirical SD much better. With huge datasets, even the naïve bootstrap’s bias shrinks, but the principled fix is to respect the dependence structure in the bootstrap.